In [1]:
import arcpy
from arcpy import env

In [2]:
import os, numpy, re
import pandas as pd

In [3]:
env.workspace = r"T:\MPO\RTP\FY20 2045 Update\Data and Resources\Network_Analysis\Network_Analysis.gdb"
env.overwriteOutput = True

In [4]:
input_folder = r"T:\MPO\RTP\FY20 2045 Update\Data and Resources"

In [5]:
EFAbound = os.path.join(input_folder, "PerformanceAnalysis", "service_transit_equity", "equity_area.shp")

In [6]:
EquityAreaID = pd.read_csv("../EquityAreaID.csv")

In [7]:
EquityAreaID.head()

,BlkGrp10,EquityArea
0,410390019043,5
1,410390020022,6
2,410390026002,10
3,410390033023,18
4,410390043001,29


In [6]:
AOI = "MPO"
service = "Jobs"
travel_mode = "Biking"
year = 2020

In [7]:
sa_layer = "SA" + travel_mode + "HH"

In [8]:
sa_layer_name = sa_layer
sa_layer = os.path.join(input_folder, sa_layer + ".shp")
target_field = 'ohh'

In [9]:
layer_for_spatial_join = "baseyear" + service + "_FeatureToPoint"

In [10]:
count_field = "ojobs"

In [10]:
spatialJoin_out_name = service + travel_mode + str(year) + "HH_SA"

In [15]:
arcpy.analysis.SpatialJoin(sa_layer, layer_for_spatial_join, spatialJoin_out_name, "JOIN_ONE_TO_ONE", "KEEP_COMMON", 'FacilityID "FacilityID" true true false 18 Double 0 18,First,#,{0},FacilityID,-1,-1;{1} "{1}" true true false 24 Double 15 23,First,#,{0},{1},-1,-1;{2} "{2}" true true false 8 Double 0 0,Sum,#,{3},{2},-1,-1'.format(sa_layer_name, target_field, count_field, layer_for_spatial_join), "COMPLETELY_CONTAINS", None, '')

<Result 'T:\\MPO\\RTP\\FY20 2045 Update\\Data and Resources\\Network_Analysis\\Network_Analysis.gdb\\MPOJobsBiking2020HH_SA'>

In [11]:
MPObound = r"V:\Data\Transportation\MPO_Boundary.shp"

In [23]:
#hh_point_layer = "baseyearHH_FeatureToPoint"
hh_point_layer = "forecastHH_FeatureToPoint"

In [13]:
input_layer = arcpy.management.SelectLayerByLocation(hh_point_layer, "COMPLETELY_WITHIN", MPObound, None, 
                                                                     "New_SELECTION", "NOT_INVERT")

In [24]:
input_layer = arcpy.management.SelectLayerByLocation(hh_point_layer, "COMPLETELY_WITHIN", EFAbound, None, 
                                                                     "NEW_SELECTION", "NOT_INVERT")

In [25]:
int(arcpy.GetCount_management(input_layer)[0])

10526

In [26]:
year = 2045

In [27]:
selected_hh_efa = "EFA_HH" + str(year)
if arcpy.Exists(selected_hh_efa):
    print("Got the selected household points in {0}...".format(str(year)))
else:
    # for the analysis in each equity-focused area
    arcpy.CopyFeatures_management(input_layer, selected_hh_efa)

In [44]:
arcpy.management.DeleteField(out_layer, target_field)

<Result 'T:\\MPO\\RTP\\FY20 2045 Update\\Data and Resources\\Network_Analysis\\Network_Analysis.gdb\\MPOJobsBiking2020HH_SA'>

In [16]:
oldFieldList = [f.name for f in arcpy.ListFields(hh_point_layer)]

In [17]:
print("The fields in the household points in {0} are {1}".format(str(year), oldFieldList))

The fields in the household points in 2020 are ['OBJECTID', 'Shape', 'ohh', 'FacilityID']


In [64]:
dropFields = [f.name for f in arcpy.ListFields(hh_point_layer)]

In [65]:
keepFields = ['OBJECTID', 'Shape',  'ohh', 'FacilityID']

In [66]:
for field in keepFields:
    dropFields.remove(field)

In [67]:
for field in dropFields:
    arcpy.management.DeleteField(input_layer, field)

In [80]:
table = arcpy.conversion.TableToTable(out_layer, '', out_layer + 't', '', '', '')

In [82]:
newField = 'FacilityID'

In [83]:
output = arcpy.AddJoin_management(input_layer, newField, table, newField, "KEEP_COMMON")
arcpy.management.CopyFeatures(output, out_layer + 'p')

<Result 'T:\\MPO\\RTP\\FY20 2045 Update\\Data and Resources\\Network_Analysis\\Network_Analysis.gdb\\MPOJobsBiking2020HH_SAp'>

In [74]:
from arcgis import GIS

In [76]:
hhsa = pd.DataFrame.spatial.from_featureclass(out_layer)

In [77]:
hhsa.shape

(72032, 6)

In [5]:
AOI = "EFA"
service = "Jobs"
travel_mode = "Biking"
year = 2020

In [14]:
import os.path
from os import path

In [15]:
outTablepath = os.path.join(input_folder, 'Network_Analysis')
SAcsv = out_layer + '.csv'
HHcsv = AOI + "_HH" + str(year) + '.csv'

In [88]:
for file in [SAcsv, HHcsv]:
    file = os.path.join(outTablepath, file)
    if path.exists(file):
        os.remove(file)

In [89]:
SAtable = arcpy.conversion.TableToTable(out_layer, outTablepath, SAcsv, '', '', '')

In [16]:
HHtable = arcpy.conversion.TableToTable(input_layer, outTablepath, HHcsv, '', '', '')

In [19]:
SDdf = pd.read_csv(os.path.join(outTablepath, SAcsv))

In [17]:
HHdf = pd.read_csv(HHtable[0])

In [94]:
SDdf.shape

(72032, 7)

In [18]:
HHdf.shape

(70678, 3)

In [21]:
SDdf.head()

,OID_,Join_Count,TARGET_FID,FacilityID,ojobs,Shape_Length,Shape_Area
0,1,26,0,11730.0,1272.219023,147314.141071,7.278085e+07
1,2,12,1,14479.0,487.174503,122858.165152,5.739519e+07
2,3,12,2,12807.0,487.174503,122858.165152,5.739519e+07
3,4,11,3,9119.0,474.177259,122775.986301,5.734810e+07
4,5,21,4,10641.0,983.769037,144011.993686,6.817592e+07


In [20]:
HHdf.head()

,OID_,ohh,FacilityID
0,267,0.965,267
1,274,0.965,274
2,276,0.965,276
3,279,0.965,279
4,280,0.965,280


In [25]:
type(HHdf[['ohh', 'FacilityID']])

pandas.core.frame.DataFrame

In [26]:
df = SDdf[['ojobs', 'FacilityID']].merge(HHdf[['ohh', 'FacilityID']], how='inner', on = 'FacilityID')

In [32]:
df.shape

(70659, 4)

In [33]:
df.head()

,Unnamed: 0,ojobs,FacilityID,ohh
0,0,2813.166214,10980.0,0.965
1,1,2324.220171,10522.0,0.965
2,2,2719.819480,11165.0,0.965
3,3,2349.664064,10545.0,0.965
4,4,3389.850014,12341.0,0.965


In [30]:
df.to_csv(os.path.join(input_folder, 'Network_Analysis', out_layer + '_df' + '.csv'))

In [31]:
df = pd.read_csv(os.path.join(input_folder, 'Network_Analysis', out_layer + '_df' + '.csv'))

In [6]:
layer_name = AOI + service + travel_mode + str(year)

In [12]:
EquityAreaID['BlkGrp10'].values[0]

410390019043

In [26]:
[i for i in EquityAreaID.index]

[0, 1, 2, 3, 4, 5]

In [14]:
BlkGrp10 = EquityAreaID['BlkGrp10'].values[0]

In [15]:
EFAbound = arcpy.management.SelectLayerByAttribute(EFAbound, "NEW_SELECTION", "BlkGrp10 = '{0}'".format(BlkGrp10), None)

In [16]:
# 2 years * 3 AOI * 2 travel modes * 2 services
sas = ["Jobs", "Amenities"]
travel_modes = ["Biking", "Walking"]
layer_names = ["baseyearJobs_FeatureToPoint", "forecastJobs_FeatureToPoint"]
AOIs = ["MPO", "EFA", "NEFA"]
# matched with layer names
jobfields = ["ojobs", "jobs"]
years = [2020, 2045]
hhfields = ["ohh", "hh"]

In [17]:
newfield = 'weighted_' + jobfields[years.index(year)]

In [19]:
fieldList = arcpy.ListFields(layer_name)
field_names = [f.name for f in fieldList]
if newfield in field_names:
    pass
else:
    arcpy.AddField_management(layer_name, newfield, "FLOAT", "", "", 50)

In [20]:
hhfield = hhfields[years.index(year)]
targetfield = layer_names[years.index(year)] + "_" + jobfields[years.index(year)]

In [21]:
arcpy.management.CalculateField(layer_name, newfield, "!{0}! * !{1}!".format(hhfield, targetfield), "PYTHON3", '', "TEXT")

<Result 'T:\\MPO\\RTP\\FY20 2045 Update\\Data and Resources\\Network_Analysis\\Network_Analysis.gdb\\EFAJobsBiking2020'>

In [22]:
input_layer = arcpy.management.SelectLayerByLocation(layer_name, "COMPLETELY_WITHIN", EFAbound, None, 
                                                                 "NEW_SELECTION", "NOT_INVERT")

In [23]:
HHsum = arcpy.da.TableToNumPyArray(input_layer, hhfield, skip_nulls=True)
WgtJobsSum = arcpy.da.TableToNumPyArray(input_layer, newfield, skip_nulls=True)

In [24]:
round(WgtJobsSum[newfield].sum() / HHsum[hhfield].sum())

8676

In [5]:
AOI = "MPO"
service = "Jobs"
travel_mode = "Biking"
year = 2020

In [6]:
if year == 2020:
    sa_layer = "SA" + travel_mode + "HH"
    point_layer = "baseyearHH_FeatureToPoint"
else:
    sa_layer = sa_layer + str(year)
    point_layer = "forecastHH_FeatureToPoint"

In [7]:
if service == "Jobs":
    if year == 2020:
        layer_for_spatial_join = "baseyear" + service + "_FeatureToPoint"
        count_field = "ojobs"
    else:
        layer_for_spatial_join = "forecast" + service + "_FeatureToPoint"
        count_field = "jobs"
else:
    layer_for_spatial_join = os.path.join(input_folder, "PerformanceAnalysis", 
                                             "service_transit_equity", "service_stops.shp")

In [9]:
oldFieldList = [f.name for f in arcpy.ListFields(sa_layer)]

In [10]:
oldField = [i for i in oldFieldList if re.search(r'FacilityID', i)][0]

In [10]:
newField = "FacilityID"

In [12]:
if oldField != newField:
    arcpy.AlterField_management(sa_layer, oldField, newField)

In [13]:
Fields = [f.name for f in arcpy.ListFields(point_layer)]

In [15]:
if newField in Fields:
    pass
else:
    arcpy.AddField_management(point_layer, newField, "LONG")

In [16]:
arcpy.CalculateField_management(point_layer, "FacilityID", "!ORIG_FID! + 1", "PYTHON3" )

<Result 'T:\\MPO\\RTP\\FY20 2045 Update\\Data and Resources\\Network_Analysis\\Network_Analysis.gdb\\baseyearHH_FeatureToPoint'>

In [23]:
table = arcpy.AddJoin_management(sa_layer, newField, point_layer, newField, "KEEP_COMMON")

In [24]:
result = arcpy.management.GetCount(table)

In [25]:
print('{} has {} records'.format(table, result[0]))

SABikingHH_Layer3 has 168 records


In [20]:
out_SAjoin = "SAJoinedTable"

In [21]:
if arcpy.Exists(out_SAjoin):
    arcpy.Delete_management(out_SAjoin)

In [22]:
arcpy.CopyFeatures_management(table, out_SAjoin)
print("Completed...")

Completed...


In [5]:
year = 2020
travel_mode = 'Biking'

In [6]:
if year == 2020:
    sa_layer = "SA" + travel_mode + "HH"
    point_layer = "baseyearHH_FeatureToPoint"
    targetField = 'ohh'
else:
    sa_layer = sa_layer + str(year)
    point_layer = "forecastHH_FeatureToPoint"
    targetField = 'hh'

In [7]:
import os, re
import pandas as pd
import geopandas as gpd

In [8]:
path = r"T:\MPO\RTP\FY20 2045 Update\Data and Resources\Network_Analysis\Network_Analysis.gdb"

In [9]:
outPath = r"T:\MPO\RTP\FY20 2045 Update\Data and Resources"

In [11]:
print("Reading the input features...")
hhSA = gpd.read_file(path, layer = sa_layer)
hhpts = gpd.read_file(path, layer = point_layer)

Reading the input features...


In [13]:
def shortenColnm(x):
    if len(x) > 10:
        charList = [char for char in x if char.isupper()]
        if len(charList) >= 5:
            colnm = ''.join(re.findall(r'[A-Z]', x))
        else:
            colnm = ''.join(re.findall(r'[A-Z]..', x))
    else:
        colnm = x
    return colnm

In [34]:
def renameCols(x):
    strList = x.split("_")
    l = len(strList)
    if l == 1:
        colnm = shortenColnm(x)
    elif l == 2:
        s = strList[1]
        if s == 'Name':
            colnm = strList[0][0:6] + s
        else:
            colnm = shortenColnm(s)       
        if strList[0] == 'shape':
            colnm = 'SHP' + s
    elif l == 3:
        colnm = shortenColnm(strList[1] + strList[2])
    else:
        print("The column name is too long.")
    return colnm

In [35]:
print("Renaming the input features...")
newField = 'FacilityID'
hhpts_df = pd.DataFrame(hhpts)
hhpts_df[newField] = hhpts_df['ORIG_FID'] + 1
hhpts_df = hhpts_df[['ohh', newField]]
hhSA = hhSA.rename(columns = {x:newField for x in list(hhSA) if re.search(r'FacilityID', x)})
# join the tables
print("Merging the input features...")
hhSAJoinedpts = hhSA.merge(hhpts_df, on=newField, how='left')

Renaming the input features...
Merging the input features...


In [36]:
# rename the columns to export due to the name length limit 10
hhSAJoinedpts.rename(columns = {x: renameCols(x) for x in list(hhSAJoinedpts)}, inplace = True)
hhSAJoinedpts_gpd = gpd.GeoDataFrame(hhSAJoinedpts)
print("Writing the output feature...")
hhSAJoinedpts_gpd.to_file(os.path.join(outPath, sa_layer + '.shp'))
print("Exported the features with the joined table!")

Writing the output feature...
Exported the features with the joined table!


In [15]:
hhpts_df['ohh'].head()

0    0.965
1    0.965
2    0.965
3    0.965
4    0.965
Name: ohh, dtype: float64

In [24]:
hhSAJoinedpts = hhSA.merge(hhpts_df, on=newField, how='left')

In [25]:
hhSAJoinedpts.columns

Index(['FacilityID', 'SAPolygons1hhqxps_Name', 'SAPolygons1hhqxps_FromBreak',
       'SAPolygons1hhqxps_ToBreak', 'ORIGObjectID', 'Facilitieshsvink_Name',
       'Facilitieshsvink_SourceID', 'Facilitieshsvink_SourceOID',
       'Facilitieshsvink_PosAlong', 'Facilitieshsvink_SideOfEdge',
       'Facilitieshsvink_CurbApproach', 'Facilitieshsvink_Status',
       'Facilitieshsvink_SnapX', 'Facilitieshsvink_SnapY',
       'Facilitieshsvink_SnapZ', 'Facilitieshsvink_DistanceToNetworkInMeters',
       'Facilitieshsvink_Attr_Length', 'Facilitieshsvink_Breaks_Length',
       'Shape_Length', 'Shape_Area', 'geometry', 'ohh'],
      dtype='object')

In [16]:
hhSAJoinedpts['ohh'].head()

0    0.965
1    0.965
2    1.930
3    0.965
4    0.965
Name: ohh, dtype: float64

In [18]:
hhSAJoinedpts_gpd['ohh'].head()

0    0.965
1    0.965
2    1.930
3    0.965
4    0.965
Name: ohh, dtype: float64

In [37]:
hhSAJoinedpts_gpd.columns

Index(['FacilityID', 'SAPolyName', 'FromBreak', 'ToBreak', 'ORIGOID',
       'FaciliName', 'SourceID', 'SourceOID', 'PosAlong', 'SideOfEdge',
       'CurApp', 'Status', 'SnapX', 'SnapY', 'SnapZ', 'DTNIM', 'AttrLength',
       'BreLen', 'Length', 'Area', 'geometry', 'ohh'],
      dtype='object')

In [1]:
import JoinHHTables as jht

In [2]:
import datetime

In [3]:
jht.JoinHHTables()

Reading the input features...
Got the data...
Renaming the input features...
Merging the input features...
Writing the output feature...
Exported the features with the joined table!
Used time: 0:01:27.256122


In [4]:
for mode in ['Biking', 'Walking']:
    for yr in [2020, 2045]:
        jht.JoinHHTables(year = yr, travel_mode = mode)
        print("Completed the join for household service area by {0} in {1}".format(mode, yr))

Reading the input features...
Got the data...
Renaming the input features...
Merging the input features...
Writing the output feature...
Exported the features with the joined table!
Used time: 0:01:31.515280
Completed the join for household service area by Biking in 2020
Reading the input features...
Got the data...
Renaming the input features...
Merging the input features...
Writing the output feature...
Exported the features with the joined table!
Used time: 0:01:30.427755
Completed the join for household service area by Biking in 2045
Reading the input features...
Got the data...
Renaming the input features...
Merging the input features...
Writing the output feature...
Exported the features with the joined table!
Used time: 0:00:46.748233
Completed the join for household service area by Walking in 2020
Reading the input features...
Got the data...
Renaming the input features...
Merging the input features...
Writing the output feature...
Exported the features with the joined table!
U

In [4]:
arcpy.management.MakeFeatureLayer("parcels_FeatureToPoint", "baseyear_hh", "ohh > 0")

<Result 'baseyear_hh'>

In [5]:
fieldList = arcpy.ListFields("baseyear_hh")

In [6]:
field_names = [f.name for f in fieldList]

In [8]:
newfield = "sa_jobs"

In [9]:
desc = arcpy.Describe("baseyear_hh")

In [10]:
fields = desc.fields

In [11]:
arcpy.ListFields("baseyear_hh", "jobs")[0].type

'Double'

In [12]:
if newfield in field_names:
    pass
else:
    arcpy.AddField_management("baseyear_hh", newfield, "FLOAT", "", "", 20)

In [45]:
service = "Jobs"
travel_mode = "Biking"
AOI = "MPO"
year = 2020

In [36]:
targetHH = arcpy.management.SelectLayerByAttribute("baseyear_hh", "NEW_SELECTION", "ORIG_FID = {0}".format(18778), None)

In [37]:
selectedSA = arcpy.management.SelectLayerByLocation("joinedtable", "COMPLETELY_CONTAINS", targetHH, 
                                               None, "NEW_SELECTION", "NOT_INVERT")

In [38]:
fieldsum = arcpy.da.TableToNumPyArray(selectedSA, "baseyearJobs_FeatureToPoint_jobs", skip_nulls=True)

In [40]:
val = fieldsum["baseyearJobs_FeatureToPoint_jobs"].sum()

In [48]:
val

73822.79370354864

In [44]:
(val*15.44)/127915.574999826

8.910751757824185

In [49]:
arcpy.management.SelectLayerByAttribute("baseyear_hh", "CLEAR_SELECTION")

id,value
0,a Layer object
1,-1


In [54]:
arcpy.analysis.SpatialJoin("baseyearHH_FeatureToPoint", "joinedtable","baseyearjobs_HH","JOIN_ONE_TO_ONE", "KEEP_COMMON",
                           'btype "btype" true true false 80 Text 0 0,First,#,baseyearHH_FeatureToPoint,btype,0,80;nrsqft "nrsqft" true true false 8 Double 0 0,First,#,baseyearHH_FeatureToPoint,nrsqft,-1,-1;rsqft "rsqft" true true false 8 Double 0 0,First,#,baseyearHH_FeatureToPoint,rsqft,-1,-1;du "du" true true false 8 Double 0 0,First,#,baseyearHH_FeatureToPoint,du,-1,-1;yrbuilt "yrbuilt" true true false 8 Double 0 0,First,#,baseyearHH_FeatureToPoint,yrbuilt,-1,-1;lpid "lpid" true true false 8 Double 0 0,First,#,baseyearHH_FeatureToPoint,lpid,-1,-1;pundev "pundev" true true false 8 Double 0 0,First,#,baseyearHH_FeatureToPoint,pundev,-1,-1;dev_land "dev_land" true true false 8 Double 0 0,First,#,baseyearHH_FeatureToPoint,dev_land,-1,-1;developed "developed" true true false 8 Double 0 0,First,#,baseyearHH_FeatureToPoint,developed,-1,-1;obtype "obtype" true true false 80 Text 0 0,First,#,baseyearHH_FeatureToPoint,obtype,0,80;orsqft "orsqft" true true false 8 Double 0 0,First,#,baseyearHH_FeatureToPoint,orsqft,-1,-1;onrsqft "onrsqft" true true false 8 Double 0 0,First,#,baseyearHH_FeatureToPoint,onrsqft,-1,-1;odu "odu" true true false 8 Double 0 0,First,#,baseyearHH_FeatureToPoint,odu,-1,-1;zid "zid" true true false 8 Double 0 0,First,#,baseyearHH_FeatureToPoint,zid,-1,-1;rezoned "rezoned" true true false 8 Double 0 0,First,#,baseyearHH_FeatureToPoint,rezoned,-1,-1;city "city" true true false 80 Text 0 0,First,#,baseyearHH_FeatureToPoint,city,0,80;annexed "annexed" true true false 8 Double 0 0,First,#,baseyearHH_FeatureToPoint,annexed,-1,-1;ozid "ozid" true true false 8 Double 0 0,First,#,baseyearHH_FeatureToPoint,ozid,-1,-1;AreaPerJob "AreaPerJob" true true false 8 Double 0 0,First,#,baseyearHH_FeatureToPoint,AreaPerJob,-1,-1;isNonRes "isNonRes" true true false 4 Long 0 0,First,#,baseyearHH_FeatureToPoint,isNonRes,-1,-1;jobs "jobs" true true false 8 Double 0 0,First,#,baseyearHH_FeatureToPoint,jobs,-1,-1;ojobs "ojobs" true true false 8 Double 0 0,First,#,baseyearHH_FeatureToPoint,ojobs,-1,-1;hh "hh" true true false 8 Double 0 0,First,#,baseyearHH_FeatureToPoint,hh,-1,-1;ohh "ohh" true true false 8 Double 0 0,First,#,baseyearHH_FeatureToPoint,ohh,-1,-1;ORIG_FID "ORIG_FID" true true false 4 Long 0 0,First,#,baseyearHH_FeatureToPoint,ORIG_FID,-1,-1;baseyearJobs_FeatureToPoint_ojobs "ojobs" true true false 8 Double 0 0,Sum,#,joinedtable,baseyearJobs_FeatureToPoint_ojobs,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,joinedtable,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,joinedtable,Shape_Area,-1,-1', 
                           "COMPLETELY_WITHIN", None, '')

<Result 'T:\\MPO\\RTP\\FY20 2045 Update\\Data and Resources\\Network_Analysis\\Network_Analysis.gdb\\baseyearjobs_HH'>

In [55]:
arcpy.analysis.SpatialJoin("baseyearHH_FeatureToPoint", "joinedtable","baseyear_jobs_HH","JOIN_ONE_TO_ONE", "KEEP_COMMON",
                           'btype "btype" true true false 80 Text 0 0,First,#,{0},btype,0,80;nrsqft "nrsqft" true true false 8 Double 0 0,First,#,{0},nrsqft,-1,-1;rsqft "rsqft" true true false 8 Double 0 0,First,#,{0},rsqft,-1,-1;du "du" true true false 8 Double 0 0,First,#,{0},du,-1,-1;yrbuilt "yrbuilt" true true false 8 Double 0 0,First,#,{0},yrbuilt,-1,-1;lpid "lpid" true true false 8 Double 0 0,First,#,{0},lpid,-1,-1;pundev "pundev" true true false 8 Double 0 0,First,#,{0},pundev,-1,-1;dev_land "dev_land" true true false 8 Double 0 0,First,#,{0},dev_land,-1,-1;developed "developed" true true false 8 Double 0 0,First,#,{0},developed,-1,-1;obtype "obtype" true true false 80 Text 0 0,First,#,{0},obtype,0,80;orsqft "orsqft" true true false 8 Double 0 0,First,#,{0},orsqft,-1,-1;onrsqft "onrsqft" true true false 8 Double 0 0,First,#,{0},onrsqft,-1,-1;odu "odu" true true false 8 Double 0 0,First,#,{0},odu,-1,-1;zid "zid" true true false 8 Double 0 0,First,#,{0},zid,-1,-1;rezoned "rezoned" true true false 8 Double 0 0,First,#,{0},rezoned,-1,-1;city "city" true true false 80 Text 0 0,First,#,{0},city,0,80;annexed "annexed" true true false 8 Double 0 0,First,#,{0},annexed,-1,-1;ozid "ozid" true true false 8 Double 0 0,First,#,{0},ozid,-1,-1;AreaPerJob "AreaPerJob" true true false 8 Double 0 0,First,#,{0},AreaPerJob,-1,-1;isNonRes "isNonRes" true true false 4 Long 0 0,First,#,{0},isNonRes,-1,-1;jobs "jobs" true true false 8 Double 0 0,First,#,{0},jobs,-1,-1;ojobs "ojobs" true true false 8 Double 0 0,First,#,{0},ojobs,-1,-1;hh "hh" true true false 8 Double 0 0,First,#,{0},hh,-1,-1;ohh "ohh" true true false 8 Double 0 0,First,#,{0},ohh,-1,-1;ORIG_FID "ORIG_FID" true true false 4 Long 0 0,First,#,{0},ORIG_FID,-1,-1;{1}_{2} "{2}" true true false 8 Double 0 0,Sum,#,joinedtable,{1}_{2},-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,joinedtable,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,joinedtable,Shape_Area,-1,-1'.format('baseyearHH_FeatureToPoint', 'baseyearJobs_FeatureToPoint', 'ojobs'), 
                           "COMPLETELY_WITHIN", None, '')

<Result 'T:\\MPO\\RTP\\FY20 2045 Update\\Data and Resources\\Network_Analysis\\Network_Analysis.gdb\\baseyear_jobs_HH'>